# GPT Prompt 생성


> **주제: 한국 보험 약관의 해석 및 비판적 분석** <br/><br/>
Topic: Interpretation and critical analysis of Korean insurance terms


# Step 2: 잠재적 '문제점 발상' 및 근거 제시 (ToT 발상 + CoT 근거, RAG 필수 활용)

### 목표

약관의 잠재적 문제점을 다양하게 발상하고, 그 근거를 제시하는 것. 이때 RAG를 적극적으로 활용해서 더 현실적이고 근거가 풍부한 문제점을 발상

### 수정 내용:

- 잠재적 문제점 발상을 시킬 때, **GPT 프롬프트에 분석 대상 약관 원문 텍스트와 함께 Step 1에서 검색된 외부 참고 자료(벡터 스토어 검색 결과)**를 반드시 포함
- 프롬프트 지침에 "제공된 약관 내용과 **추가 참고 자료**를 꼼꼼히 비교하며 문제점을 발상하라"고 명시
- 특히 문제점의 '근거 및 추론 과정' (CoT) 부분을 설명할 때, 약관 원문의 어떤 문구와 더불어 추가 참고 자료의 어떤 내용을 참고해서 그 문제점을 발상하게 되었는지 구체적으로 언급하도록 유도. (예: "약관 제X조 Y항 내용과 금감원 분쟁 사례집 Z 페이지의 유사 사례를 보니 이런 문제가 발생할 수 있습니다.")
- 이전 약관 전체 요약 내용도 여전히 컨텍스트로 함께 제공해서 전체 맥락 따름.
- ToT 발상 단계이므로 temperature는 좀 높게 유지, Self-consistency로 여러 후보를 생성하고 안정적인 목록을 선택하는 것은 동일 적용.

In [1]:
!pip install -U langchain-community langchain-openai chromadb PyMuPDF langchain-chroma

In [2]:
import os
import json
import re
import openai
!pip install FPDF
from fpdf import FPDF
# Google Drive 마운트 필요
from google.colab import drive
from google.colab import userdata
import time
drive.mount('/content/drive')
from collections import defaultdict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings # <-- 이 임포트 라인 사용
from langchain_openai import ChatOpenAI     # <-- 이 임포트 라인 사용
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document

  Preparing metadata (setup.py) ... done
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d087d39050b8431352f4126e74294f5221e07007741b6161b9a358abcd53c5d3
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built FPDF
Mounted at /content/drive


In [3]:
# --- OpenAI API 키 설정 (userdata로 가져와 os.environ에 설정) ---
print("OpenAI API 키 설정 확인 중 (userdata 사용)...")
# Colab '비밀' 저장소에서 'OPENAI_API_KEY' 값을 가져옵니다.
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = xx
client = openai.OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

OpenAI API 키 설정 확인 중 (userdata 사용)...


In [4]:
# 임베딩 모델 로드 (벡터 스토어 로드 및 RAG 검색 시 필요)
try:
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # 환경 변수 사용
    print("OpenAI 임베딩 모델 준비 완료.")
except Exception as e:
    print(f"!!! 오류: 임베딩 모델 로드 오류: {e} !!!")
    print("환경 변수에 설정된 API 키가 유효한지, 모델 이름이 올바른지 확인하세요.")
    embeddings = None

# GPT 모델 준비 (심층 이해 분석 및 CoT 추론 시 필요)
try:
    llm = ChatOpenAI(model="gpt-4o-2024-05-13", temperature=0.0) # 환경 변수 사용
    print(f"GPT 모델 준비 완료: {llm.model_name}")
except Exception as e:
    print(f"!!! 오류: GPT 모델 로드 오류: {e} !!!")
    print("환경 변수에 설정된 API 키가 유효한지, 모델 이름이 올바른지 확인하세요.")
    llm = None

OpenAI 임베딩 모델 준비 완료.
GPT 모델 준비 완료: gpt-4o-2024-05-13


In [5]:
# --- 벡터 스토어 영구 저장 경로 지정 --- (이전 코드와 동일)
PERSIST_DIRECTORY = '/content/drive/MyDrive/PROJECT/cn_api_project/chroma_db' # 예시 경로

# --- 벡터 스토어 로드 (최신 Chroma 클래스 사용) ---
rag_vectorstore = None # 초기화
# 임베딩 모델이 성공적으로 로드되었고 (API 키가 설정되었고) 저장 경로가 존재하면 로드 시도
if embeddings is not None and os.path.exists(PERSIST_DIRECTORY):
    print(f"\n--- 벡터 스토어 로드 중: '{PERSIST_DIRECTORY}' ---")
    try:
        # 영구 저장된 ChromaDB에서 벡터 스토어 로드 (최신 Chroma 클래스 사용)
        # 최신 Chroma 클래스도 로드 시 persist_directory와 embedding_function을 받습니다.
        rag_vectorstore = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)
        print("벡터 스토어 로드 완료.")
        print(f"로드된 청크 개수: {rag_vectorstore._collection.count()}") # 로드된 청크 개수 확인!

    except Exception as e:
        print(f"!!! 오류: 벡터 스토어 로드 중 오류 발생 !!!")
        print(f"오류 내용: {e}")
        print(f"경로 확인: '{PERSIST_DIRECTORY}'에 파일 있는지, 로드 시 사용한 임베딩 모델이 올바른지 확인하세요.")
        rag_vectorstore = None
else:
    if embeddings is None:
        print("\n!!! 벡터 스토어 로드 불가: 임베딩 모델 로드 오류 (API 키 확인 필요). !!!")
    elif not os.path.exists(PERSIST_DIRECTORY):
         print(f"\n!!! 벡터 스토어 로드 불가: 저장 폴더를 찾을 수 없습니다 - '{PERSIST_DIRECTORY}' !!!")
         print("첫 번째 노트북에서 Step 0을 실행하여 벡터 스토어를 해당 경로에 저장했는지 확인해주세요.")



--- 벡터 스토어 로드 중: '/content/drive/MyDrive/PROJECT/cn_api_project/chroma_db' ---
벡터 스토어 로드 완료.
로드된 청크 개수: 37249


In [7]:
# --- 2단계 - 잠재적 문제점 발상 프롬프트 템플릿 (보충된 버전) ---
problem_sampling_template_rag_step_2 = """

## 역할:
당신은 소비자의 권익을 보호하기 위해 보험 약관의 위험 요소를 깊이 탐색하고 분석하는 **전문 분석가**입니다.
당신의 임무는 약관의 잠재적인 함정, 허점, 불합리한 구조를 **외부 참고 자료와 함께 통합적으로 분석**하여 **소비자가 피해를 입을 수 있는 위험요소를 창의적으로 발굴**하는 것입니다.

---

## 🎯 목표:
제공된 다음 세 가지 자료를 **비교·통합**하여,
▶ 분석 대상 약관 텍스트:
{analysis_target_text}

▶ 이전 약관 요약:
{previous_full_summary}

▶ 참고 자료 (법령, 분쟁사례 등):
{rag_context}

소비자가 **오해하거나 피해를 입을 수 있는 잠재적 문제점**을 **최소 7건 이상**, **창의적이며 다각도로** 도출하세요.

---

## 🔍 분석 방식:

### 1. 비교·비판적 시각 유지:
- 분석 대상 약관의 **문구/표현/용어/조항 구조**를 세밀하게 분석하세요.
- **이전 약관 요약**을 통해 맥락과 흐름, 변화 포인트를 파악하세요.
- **참고자료**와 **적극 비교**하여 아래와 같은 기준으로 문제점을 탐색하세요:
  - 판례나 금감원 사례와의 모순
  - 유사 보험상품과의 보장 격차
  - 현실 의료 상황과의 괴리
  - 법률/감독규정과의 불일치

### 2. 다양한 관점에서 잠재적 위험을 ToT 방식으로 탐색하세요:
- 약관 문구의 **모호성/다의성**
- 보장 및 면책 범위의 **불명확성**
- **고지/통지 의무 조항의 부담**
- 보험금 청구 및 지급 절차의 **불합리성**
- **상충되는 조항 간의 충돌 가능성**
- 법령/판례와의 불일치
- 의료 현실과 괴리된 조항
- 소비자에게 중요한 정보의 은폐 또는 비노출
- 기타 예상 외의 소비자 피해 시나리오

### 3. 각 문제점은 다음의 **CoT 방식 논리적 근거 구조**를 따르세요:

| 항목 | 내용 |
|------|------|
| (A) 분석 대상 약관 문구 | 해당 문제의 핵심이 되는 문구 직접 인용 |
| (B) 비교한 참고자료 | 판례, 법령, 유사 약관, 금감원 사례 등 |
| (C) 문제 도출 논리 | 비교를 통해 도출된 문제의 구체적 이유 |
| (D) 예상 시나리오 | 현실에서 어떻게 소비자 피해로 이어질 수 있는지 |

---

## ⚠️ 필수 요건:
- **각 문제점마다 반드시 [추가 참고자료] 1건 이상 인용**해야 합니다.
- **단순 해석이 아니라 비교와 근거 기반의 창의적 추론**이 포함되어야 합니다.
- "문제점_유형"은 다음 중 복수 선택 가능:
  * 모호한 표현
  * 보상 제외 조건의 불명확성
  * 고지 의무/통지 의무
  * 지급/청구 절차
  * 문서 내 상충 조항
  * 법령/판례와의 불일치
  * 현실과의 괴리
  * 소비자에게 불리한 정보 미노출
  * 기타

---

## [선택 사항] 신뢰도 등급:
- 각 문제점에 대해 다음 중 하나의 신뢰도 등급을 제시할 수 있습니다:
  * 신뢰 (반복되는 실제 사례 있음)
  * 주의 (근거는 있으나 해석에 따라 다름)
  * 불확실 (가능성은 있으나 근거가 약함)

---

## ✅ 출력 포맷 (JSON):

```json
{{
  "잠재적_문제점_후보": [
    {{
      "id": 1,
      "간단_표시용_요약": "정신질환 입원 보장 제외로 인한 사각지대",
      "문제점_요약": "정신질환 입원 불인정 조항",
      "발생_가능_시나리오": "우울증으로 입원한 가입자가 보험금을 청구했으나, 해당 조항에 따라 지급 거부됨",
      "근거_및_추론과정": "약관 제6조 3항의 '정신질환으로 인한 입원은 보험금 지급 대상에서 제외'라는 문구는, 금감원 분쟁사례집 2023년 p.47의 사례에서 유사 분쟁이 반복됨. 의료 현실에서는 우울증 등의 장기 입원 증가 추세인데, 약관은 이를 전면 보장 제외하여 보장의 실익이 없음.",
      "관련_약관_원문_표현": "정신질환으로 인한 입원은 보험금 지급 대상에서 제외",
      "문제점_유형": ["보상 제외 조건의 불명확성", "현실과의 괴리"],
      "예상_소비자_영향": "보험금 전액 미지급",
      "참고_자료_출처": ["금감원 분쟁사례집 2023 p.47", "보험업감독규정 제9조"],
      "근거_신뢰도": "신뢰"
    }}
    // ...more 문제점...
  ]
}}
"""

In [8]:
def request_gpt(message, model="gpt-4o-2024-05-13", temperature=0.7, type="json_object"):
# 이 함수는 openai.OpenAI() 클라이언트 객체를 사용합니다.
# client 객체가 위에 정의되어 있다고 가정합니다.
    if client is None:
        print("API 클라이언트가 초기화되지 않았습니다.")
        return {} if type == "json_object" else "API 클라이언트 오류"

    messages = [{"role": "user", "content": message}]
    try:
        response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                response_format={"type" : type}
            ).model_dump()
        content = response['choices'][0]['message']['content']
        if type == "json_object":
            content = content.strip()
            # 가끔 ```json ... ``` 형태로 오는 응답 처리
            if content.startswith("```json"): content = content[7:]
            if content.endswith("```"): content = content[:-3]
            content = content.strip()
            return json.loads(content)
        else: return content
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return {} if type == "json_object" else f"API Error: {e}"

In [9]:
def identify_potential_problems_with_rag(analysis_target_text, previous_full_summary, rag_vectorstore, iterations=7, model="gpt-4o-2024-05-13", temperature=0.7):
    if client is None:
        print("\n!!! API 클라이언트가 초기화되지 않아 문제점 발상을 실행할 수 없습니다. !!!")
        return []
    # RAG 검색은 rag_vectorstore가 None이 아니면 시도합니다.
    rag_context_text = "참고 자료 검색 실패 또는 없음." # 초기값 설정
    if rag_vectorstore is not None:
        print("\n--- RAG 검색 시작 ---")
        try:
            retriever = rag_vectorstore.as_retriever(search_kwargs={"k": 5}) # 관련 문서 청크 5개 검색 (조절 가능)

            # RAG 검색 실행: 분석 대상 텍스트를 쿼리로 사용
            # 분석 대상 텍스트 전체가 너무 길 수 있으므로 일부만 쿼리로 사용 (예: 앞 500자)
            query_text_for_rag = analysis_target_text[:500] # 쿼리 길이를 제한하여 검색 (조절 필요)
            if not query_text_for_rag.strip():
                print("경고: 분석 대상 텍스트가 비어있거나 너무 짧아 RAG 검색 쿼리 생성 불가.")
                rag_context_text = "분석 대상 텍스트 부족: 참고 자료 검색 불가."
            else:
                retrieved_docs = retriever.invoke(query_text_for_rag)
                rag_context_text = "\n----\n".join([
                    f"출처: {doc.metadata.get('source', '알 수 없음')}, 페이지: {doc.metadata.get('page', '알 수 없음')}\n{doc.page_content}"
                    for doc in retrieved_docs
                ])
                print(f"RAG 검색 완료. 총 {len(retrieved_docs)}개의 참고 자료 확보.")
                # print(f"참고 자료 일부:\n{rag_context_text[:500]}...") # 디버깅용

        except Exception as e:
            print(f"!!! 오류: RAG 검색 중 오류 발생: {e} !!!")
            print("벡터 스토어 로드 상태, 임베딩 모델, API 키 등을 확인하세요.")
            rag_context_text = "오류 발생: 참고 자료 검색 실패."

    # Self-consistency를 위한 발상 반복
    problem_list_counts = defaultdict(int)
    problem_list_details = {}

    print (f"\n===== [2단계] 잠재적 문제점 발상 Self-consistency (RAG 활용) 분석 시작 ({iterations}회 반복) =====")

    sampling_temperature = temperature # 발상 온도 사용 (보통 0.7 ~ 1.0)

    for i in range(iterations):
        print(f"\n----- [{i+1}/{iterations} 번째 문제점 발상 시도 중] -----")

        # 프롬프트 메시지 구성 (보충된 템플릿 사용)
        message = problem_sampling_template_rag_step_2.format(
            previous_full_summary=previous_full_summary if previous_full_summary else "없음",
            analysis_target_text=analysis_target_text, # 분석 대상 텍스트 전체 전달
            rag_context=rag_context_text if rag_context_text else "참고 자료 없음." # RAG 검색 결과 전달!
        )

        try:
            # request_gpt 함수 사용 (client 객체 사용)
            response_json = request_gpt(
                message,
                model=model,
                temperature=sampling_temperature,
                type="json_object" # JSON 형태로 응답 요청
            )

            # 응답 형식이 기대와 다를 경우 처리
            if not isinstance(response_json, dict) or "잠재적_문제점_후보" not in response_json:
                print("문제점 발상 응답 형식이 올바르지 않습니다. 결과 스킵.")
                print(f"Raw Response Sample: {str(response_json)[:200]}...")
                continue # 이번 결과는 무시

            problem_candidates = response_json.get("잠재적_문제점_후보", [])

            # 문제점 후보 목록을 문자열로 변환하여 일관성 체크 기준 마련 (JSON 정규화)
            problem_list_string_key = json.dumps(problem_candidates, sort_keys=True, ensure_ascii=False)

            # 결과가 비어있거나 너무 적으면 스킵
            if not problem_candidates or len(problem_candidates) < 3: # 최소 후보 개수 기준 (조절 필요)
                print(f"발상된 문제점 후보 개수가 부족합니다 ({len(problem_candidates)}개). 결과 스킵.")
                continue

            # print(f"발상된 문제점 후보 목록 (총 {len(problem_candidates)}개) 생성 완료.") # 디버깅용

            problem_list_counts[problem_list_string_key] += 1
            if problem_list_string_key not in problem_list_details:
                problem_list_details[problem_list_string_key] = problem_candidates

        except Exception as e:
            print(f"API 호출 중 오류 발생 또는 JSON 처리 오류: {e}")
            error_key = f"[API 오류: {e}]"
            problem_list_counts[error_key] += 1


    # Self-consistency 결과 요약 및 최빈값 선택
    print(f"\n===== [2단계] 잠재적 문제점 발상 Self-consistency (RAG 활용) 결과 요약 빈도 =====")
    if not problem_list_counts:
        print("문제점 발상 결과가 없습니다.")
        return []

    sorted_results = sorted(problem_list_counts.items(), key=lambda x: x[1], reverse=True)
    for key_string, count in sorted_results:
        display_key = key_string[:100].replace('\n', ' ') + '...' if len(key_string) > 100 else key_string
        print(f"'{display_key}': {count}회")

    most_frequent_list_key = sorted_results[0][0]

    print(f"\n===== [2단계] Self-consistency 최종 문제점 발상 결과 (RAG 활용) =====")
    final_problem_candidates_list = problem_list_details.get(most_frequent_list_key, [])
    print(f"총 {len(final_problem_candidates_list)}개의 문제점 후보 선정 (Self-consistency 기반):")
    for p in final_problem_candidates_list:
        # f-string 오류 방지를 위해 replace('\n', ' ') 결과를 변수에 먼저 저장 후 사용
        근거_CoT_display = p.get('근거_및_추론과정', 'N/A')[:150].replace('\n', ' ')
        print(f"- 요약: {p.get('문제점_요약', 'N/A')}")
        print(f"  근거 (CoT): {근거_CoT_display}...") # 수정된 부분
        print(f"  관련 약관/자료: {p.get('관련_약관_원문_표현', 'N/A')} | 출처: {p.get('참고_자료_출처', 'N/A')}")
        print("-" * 20)

    return final_problem_candidates_list

In [19]:
from collections import defaultdict
import json

def perform_rag_search(query_text, rag_vectorstore, k=5):
    if not rag_vectorstore:
        return "참고 자료 검색 실패 또는 없음.", []

    try:
        retriever = rag_vectorstore.as_retriever(search_kwargs={"k": k})
        retrieved_docs = retriever.invoke(query_text)
        rag_context = "\n----\n".join([
            f"출처: {doc.metadata.get('source', '알 수 없음')}, 페이지: {doc.metadata.get('page', '알 수 없음')}\n{doc.page_content}"
            for doc in retrieved_docs
        ])
        print(f"RAG 검색 완료: {len(retrieved_docs)}개 문서 확보.")
        return rag_context, retrieved_docs
    except Exception as e:
        print(f"!!! RAG 검색 오류: {e} !!!")
        return "오류 발생: 참고 자료 검색 실패.", []

def identify_potential_problems_with_rag(analysis_target_text, previous_full_summary, rag_vectorstore,
                                         iterations=5, model="gpt-4o-2024-05-13", temperature=0.7):
    if client is None:
        print("\n[오류] API 클라이언트가 초기화되지 않았습니다.")
        return []

    print("\n===== [2단계] 문제점 발상 Self-consistency (RAG 활용) 시작 =====")

    # RAG 검색 수행
    query_text = analysis_target_text[:500].strip()
    if not query_text:
        print("경고: 분석 대상 텍스트가 비어 있어 RAG 검색 불가.")
        rag_context_text = "분석 대상 텍스트 부족: 참고 자료 검색 불가."
        retrieved_docs = []
    else:
        rag_context_text, retrieved_docs = perform_rag_search(query_text, rag_vectorstore)

    # 문제점 후보 수집
    problem_list_counts = defaultdict(int)
    problem_list_details = {}

    for i in range(iterations):
        print(f"\n----- [{i+1}/{iterations}] 문제점 발상 시도 -----")

        message = problem_sampling_template_rag_step_2.format(
            previous_full_summary=previous_full_summary or "없음",
            analysis_target_text=analysis_target_text,
            rag_context=rag_context_text or "참고 자료 없음."
        )

        try:
            response_json = request_gpt(
                message, model=model,
                temperature=temperature, type="json_object"
            )

            candidates = response_json.get("잠재적_문제점_후보", [])
            if not isinstance(candidates, list) or len(candidates) < 3:
                print(f"[스킵] 유효한 문제점 후보 부족 ({len(candidates)}개).")
                continue

            key = json.dumps(candidates, sort_keys=True, ensure_ascii=False)
            problem_list_counts[key] += 1
            if key not in problem_list_details:
                problem_list_details[key] = candidates

        except Exception as e:
            print(f"[오류] GPT 호출 실패 또는 JSON 오류: {e}")
            error_key = f"[API 오류: {e}]"
            problem_list_counts[error_key] += 1

    # 결과 요약 및 최빈값 선정
    print("\n===== [2단계] Self-consistency 발상 결과 요약 =====")
    if not problem_list_counts:
        print("문제점 후보 없음.")
        return []

    sorted_results = sorted(problem_list_counts.items(), key=lambda x: x[1], reverse=True)
    for key, count in sorted_results:
        summary = key[:100].replace('\n', ' ')
        print(f"'{summary}...': {count}회")

    best_key = sorted_results[0][0]
    final_candidates = problem_list_details.get(best_key, [])

    print(f"\n===== 최종 선정된 문제점 후보 ({len(final_candidates)}개) =====")
    for p in final_candidates:
        근거_CoT = p.get('근거_및_추론과정', 'N/A')[:150].replace('\n', ' ')
        print(f"- 요약: {p.get('문제점_요약', 'N/A')}")
        print(f"  근거 (CoT): {근거_CoT}...")
        print(f"  관련 약관/자료: {p.get('관련_약관_원문_표현', 'N/A')} | 출처: {p.get('참고_자료_출처', 'N/A')}")
        print("-" * 20)

    return final_candidates

In [20]:
def run_problem_identification_step(analysis_text, previous_summary, use_rag=False, rag_vectorstore=None, iterations=7, model="gpt-4o-2024-05-13", temperature=0.7):
    if use_rag and rag_vectorstore is not None:
        print("\n--- [2단계] RAG를 사용한 잠재적 문제점 발상 시작 ---")
        problem_candidates = identify_potential_problems_with_rag(
        analysis_target_text=analysis_text,
        previous_full_summary=previous_summary,
        rag_vectorstore=rag_vectorstore,
        iterations=iterations,
        model=model,
        temperature=temperature # 발상 온도
        )
    else:
        print("\n--- [2단계] 일반 잠재적 문제점 발상 시작 (RAG 비활성) ---")
        problem_candidates = identify_potential_problems_with_rag(
        analysis_target_text=analysis_text,
        previous_full_summary=previous_summary,
        rag_vectorstore=None, # RAG 비활성 시 None 전달
        iterations=iterations,
        model=model,
        temperature=temperature
        )
    print(f"\n--- [2단계] 잠재적 문제점 발상 완료. 총 {len(problem_candidates)}개의 후보 선정. ---")
    return problem_candidates

In [21]:
# full_policy_text_file_path = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_애니펫.txt'

In [22]:
full_policy_text_file_path = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_의기냥냥.txt'

In [23]:
analysis_target_text_full = "" # 변수 초기화
print(f"\n--- 분석 대상 약관 원문 전체 텍스트 로드 중: '{full_policy_text_file_path}' ---")
try: # 파일 존재 확인
    if os.path.exists(full_policy_text_file_path):
    # 파일 읽기 모드('r', read)로 열고 전체 내용 읽기
        with open(full_policy_text_file_path, 'r', encoding='utf-8') as f:
            analysis_target_text_full = f.read()
        print("약관 원문 전체 텍스트 로드 완료.")
        print(f"로드된 텍스트 길이: {len(analysis_target_text_full)} 문자.")
    else:
        print(f"!!! 오류: 약관 원문 전체 텍스트 파일을 찾을 수 없습니다 - '{full_policy_text_file_path}' !!!")
        print("Google Drive 마운트 상태와 경로를 확인해주세요.")
        analysis_target_text_full = "" # 파일 없으면 빈 문자열

except Exception as e:
    print(f"!!! 오류: 약관 원문 전체 텍스트 로드 중 오류 발생: {e} !!!")
    analysis_target_text_full = "" # 로드 실패 시 빈 문자열


--- 분석 대상 약관 원문 전체 텍스트 로드 중: '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_의기냥냥.txt' ---
약관 원문 전체 텍스트 로드 완료.
로드된 텍스트 길이: 357323 문자.


In [ ]:
## previous_summary_file_path = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_의기냥냥_summary.txt'

In [24]:
previous_summary_file_path = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/삼성화재_반려묘보험_의기냥냥_summary.txt'

In [25]:
previous_full_summary_text = "" # 변수 초기화
print(f"\n--- 이전 약관 전체 요약 텍스트 로드 중: '{previous_summary_file_path}' ---")
try:
    # 파일 존재 확인
    if os.path.exists(previous_summary_file_path):
    # 파일 읽기 모드('r', read)로 열고 전체 내용 읽기
        with open(previous_summary_file_path, 'r', encoding='utf-8') as f:
            previous_full_summary_text = f.read()
            print("이전 약관 전체 요약 로드 완료.")
            print(f"로드된 텍스트 길이: {len(previous_full_summary_text)} 문자.")
    else:
        print(f"!!! 경고: 이전 약관 전체 요약 파일을 찾을 수 없습니다 - '{previous_summary_file_path}' !!!")
        print("이전 약관 요약 없이 문제점 발상이 진행됩니다.")
        previous_full_summary_text = "" # 파일 없으면 빈 문자열

except Exception as e:
    print(f"!!! 오류: 이전 약관 전체 요약 텍스트 로드 중 오류 발생: {e} !!!")
    previous_full_summary_text = "" # 로드 실패 시 빈 문자열


--- 이전 약관 전체 요약 텍스트 로드 중: '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/삼성화재_반려묘보험_의기냥냥_summary.txt' ---
이전 약관 전체 요약 로드 완료.
로드된 텍스트 길이: 173890 문자.


### 2단계 실행 예시 (문제점 발상)



> 벡터 스토어가 로드되었고 (rag_vectorstore is not None)
GPT 모델이 로드되었다면 (llm is not None)
그리고 분석 대상 텍스트와 이전 요약 텍스트가 로드되었다면
2단계 분석을 실행





> 1. 약관 원문 전체 텍스트 분할: analysis_target_text_full 변수에 담긴 약관 원문 전체 텍스트를 적절한 크기(예: 3000~5000자)의 청크로 나눠. 이때 약관의 목차나 조항 기준으로 자르면 더 의미있는 단위가 될 수 있지만, 간단하게는 문자열 길이 기준으로 자를 수 있어.
2. 분할된 각 청크 순회: 나눠진 각 청크에 대해 반복문을 돌면서 문제점 발상 Self-consistency (run_problem_identification_step)를 실행해.
3. 프롬프트 내용: 각 반복 시 run_problem_identification_step 함수에 다음 내용을 전달해.
analysis_text: 현재 반복에서 처리할 약관 원문 텍스트 청크 (전체 대신 부분!).
previous_summary: 이전 약관 전체 요약 (이건 계속 전체를 넘겨줘도 괜찮을 거야).
rag_vectorstore: 벡터 스토어 객체 (RAG 검색에 사용).
4. 결과 취합: 각 청크별로 발상된 문제점 후보 목록 결과들을 하나의 큰 리스트로 모아.



In [26]:
if rag_vectorstore is not None and llm is not None and analysis_target_text_full and previous_full_summary_text: # 입력 데이터도 확인
    print(f"\n--- 2단계 잠재적 문제점 발상 시작 (약관 원문 전체 대상, RAG 활용) ---")


--- 2단계 잠재적 문제점 발상 시작 (약관 원문 전체 대상, RAG 활용) ---


In [27]:
# --- 약관 원문 전체 텍스트를 청크로 분할 ---
chunk_size_analysis = 2000 # 분석 프롬프트에 넣을 텍스트 청크 크기 (토큰 고려 조절)
                               # gpt-4o 128k 토큰 감안, RAG 검색 결과 등 포함 시 안전하게 4000-8000자 추천
chunk_overlap_analysis = 200 # 청크 간 겹침 (컨텍스트 유지)

# Langchain RecursiveCharacterTextSplitter를 다시 사용하여 분할
# 임베딩 시 사용한 것과 동일하거나 다른 설정 사용 가능
analysis_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_analysis,
    chunk_overlap=chunk_overlap_analysis
        # chunk_size와 overlap 설정은 RAG 임베딩 시와 다를 수 있습니다.
        # 여기서는 분석 프롬프트에 들어갈 텍스트 양을 조절하는 목적입니다.
    )

# Document 객체로 만들어야 TextSplitter 사용 가능. 간단하게 전체 텍스트를 하나의 Document로 만듭니다.
# 메타데이터는 필요에 따라 추가 (예: source)
from langchain_core.documents import Document # 임포트 확인
doc_to_split = Document(page_content=analysis_target_text_full, metadata={"source": "원본 약관 전체"})

print(f"\n약관 원문 전체 텍스트 ({len(analysis_target_text_full)}자)를 {chunk_size_analysis}자씩 분할 중...")
analysis_chunks = analysis_text_splitter.split_documents([doc_to_split]) # split_documents는 Document 리스트를 받습니다.
print(f"약관 원문 전체 텍스트 분할 완료. 총 {len(analysis_chunks)}개의 분석 청크 생성.")


약관 원문 전체 텍스트 (357323자)를 2000자씩 분할 중...
약관 원문 전체 텍스트 분할 완료. 총 199개의 분석 청크 생성.


In [28]:
all_problem_candidates_from_chunks = [] # 모든 청크에서 발상된 문제점 후보들을 모을 리스트

# --- 분할된 각 청크에 대해 문제점 발상 실행 ---
for i, chunk_doc in enumerate(analysis_chunks):
    current_analysis_chunk_text = chunk_doc.page_content # 현재 청크의 텍스트 내용
    print(f"\n\n===== [전체 약관 분석] 청크 {i+1}/{len(analysis_chunks)} 문제점 발상 시작 =====")

    # run_problem_identification_step 함수 호출하여 현재 청크에 대해 2단계 실행
    # analysis_text에 현재 청크 텍스트를 전달
    # previous_summary에 이전 약관 전체 요약 텍스트를 전달
    # use_rag=True로 설정하여 RAG 활성화 (각 청크 분석 시마다 관련 RAG 검색 수행)
    chunk_problem_candidates = run_problem_identification_step(
        analysis_text=current_analysis_chunk_text, # 분석 대상: 현재 약관 청크!
        previous_summary=previous_full_summary_text, # 이전 요약 내용 전체! (각 청크 분석 시마다 참고)
        use_rag=True, # RAG 활성화!
        rag_vectorstore=rag_vectorstore, # 로드된 벡터 스토어 전달
        iterations=1, # Self-consistency 반복 횟수 (각 청크별 반복) - 전체 실행 시간 고려 조절 필요!
        model="gpt-4o-2024-05-13", # GPT 모델
        temperature=1.0 # 발상 단계 온도
        )

    # 현재 청크에서 발상된 문제점 후보들을 전체 리스트에 추가
    all_problem_candidates_from_chunks.extend(chunk_problem_candidates)

    print(f"\n--- 청크 {i+1}/{len(analysis_chunks)} 문제점 발상 완료. 현재까지 발상된 총 후보 개수: {len(all_problem_candidates_from_chunks)} ---")



===== [전체 약관 분석] 청크 1/199 문제점 발상 시작 =====

--- [2단계] RAG를 사용한 잠재적 문제점 발상 시작 ---

===== [2단계] 문제점 발상 Self-consistency (RAG 활용) 시작 =====
RAG 검색 완료: 5개 문서 확보.

----- [1/1] 문제점 발상 시도 -----

===== [2단계] Self-consistency 발상 결과 요약 =====
'[{"id": 1, "간단_표시용_요약": "고지의무 위반 시 불리한 해석", "관련_약관_원문_표현": "계약자 또는 피보험자가 고의 또는 중대한 과실로 제1항을 위반한 경우 회...': 1회

===== 최종 선정된 문제점 후보 (7개) =====
- 요약: 고지의무 위반 시 보험사의 해지권 과다
  근거 (CoT): 약관 제18조에서는 '고지 의무 위반의 효과'로 보험사의 해지권을 명시하고 있음. 금감원 분쟁사례집에서는 경미한 병력 미고지로 인해 보험사의 해지권 남용이 발생한 사례가 다수 존재함....
  관련 약관/자료: 계약자 또는 피보험자가 고의 또는 중대한 과실로 제1항을 위반한 경우 회사는 계약을 해지할 수 있다. | 출처: ['금감원 분쟁사례집 2023 p.28', '보험업감독규정 제9조']
--------------------
- 요약: 보험금 지급 제외 사유의 모호성
  근거 (CoT): 약관 제5조에서는 '보험금을 지급하지 않는 사유'에 관한 문구가 매우 포괄적이며 모호함. 예를 들어 '회사의 판단에 따라 보험금을 지급하지 않을 수 있다'는 표현이 있음. 이는 금감원 분쟁사례집 2022-4, p. 73의 사례에서도 문제가 되었음....
  관련 약관/자료: 회사의 판단에 따라 보험금을 지급하지 않을 수 있다. | 출처: ['금감원 분쟁사례집 2022-4, p. 73']
--------------------
- 요약: 자연재해 피해 보장 제외 조건
  근거 (CoT): 약관 제5조에서 '전쟁, 폭동, 자연재해로 인한 손해는 보상하지

KeyboardInterrupt: 

In [30]:
print(all_problem_candidates_from_chunks)

[{'id': 1, '간단_표시용_요약': '고지의무 위반 시 불리한 해석', '문제점_요약': '고지의무 위반 시 보험사의 해지권 과다', '발생_가능_시나리오': '계약자가 경미한 병력을 고지하지 않은 것을 이유로 보험사에서 계약을 해지함', '근거_및_추론과정': "약관 제18조에서는 '고지 의무 위반의 효과'로 보험사의 해지권을 명시하고 있음. 금감원 분쟁사례집에서는 경미한 병력 미고지로 인해 보험사의 해지권 남용이 발생한 사례가 다수 존재함.", '관련_약관_원문_표현': '계약자 또는 피보험자가 고의 또는 중대한 과실로 제1항을 위반한 경우 회사는 계약을 해지할 수 있다.', '문제점_유형': ['고지 의무/통지 의무', '법령/판례와의 불일치'], '예상_소비자_영향': '계약 해지 및 보험금 미지급', '참고_자료_출처': ['금감원 분쟁사례집 2023 p.28', '보험업감독규정 제9조'], '근거_신뢰도': '신뢰'}, {'id': 2, '간단_표시용_요약': '모호한 표현으로 인한 소비자 혼란', '문제점_요약': '보험금 지급 제외 사유의 모호성', '발생_가능_시나리오': '소비자가 상해로 입원 후 보험금을 청구했더니 보험금 지급 제외 사유가 모호해 결과를 예측할 수 없었음', '근거_및_추론과정': "약관 제5조에서는 '보험금을 지급하지 않는 사유'에 관한 문구가 매우 포괄적이며 모호함. 예를 들어 '회사의 판단에 따라 보험금을 지급하지 않을 수 있다'는 표현이 있음. 이는 금감원 분쟁사례집 2022-4, p. 73의 사례에서도 문제가 되었음.", '관련_약관_원문_표현': '회사의 판단에 따라 보험금을 지급하지 않을 수 있다.', '문제점_유형': ['모호한 표현', '보상 제외 조건의 불명확성'], '예상_소비자_영향': '보험금 지급 지연 또는 미지급', '참고_자료_출처': ['금감원 분쟁사례집 2022-4, p. 73'], '근거_신뢰도': '주의'}, {'id': 3, '간단_표시용_요약': '자연재해로 인한 피해 보장 제외

In [29]:
all_problem_candidates_from_chunks = list(set(all_problem_candidates_from_chunks)) # 중복 제거

TypeError: unhashable type: 'dict'

In [ ]:
print("\n\n===== 2단계 최종 결과 (전체 약관 문제점 발상 종합) =====")
print(f"약관 원문 전체 분석 결과, 총 {len(all_problem_candidates_from_chunks)}개의 잠재적 문제점 후보 발상.")



===== 2단계 최종 결과 (전체 약관 문제점 발상 종합) =====
약관 원문 전체 분석 결과, 총 227개의 잠재적 문제점 후보 발상.


### gdrive에 step2 폴더 만들어 저장

In [31]:
def generate_step2_result_filename(full_policy_text_file_path: str, prefix: str = 'problems_', extension: str = '.json') -> str:
    """
    주어진 약관 파일 경로에서 파일명을 추출하여 'problems_' 접두사와 함께 결과 파일명을 생성합니다.

    Args:
        full_policy_file_path (str): 원본 약관 파일의 전체 경로 (예: '/.../한화_스마일펫보험.txt')
        prefix (str): 결과 파일의 접두사 (기본값 'problems_')
        extension (str): 결과 파일의 확장자 (기본값 '.json')

    Returns:
        str: 생성된 결과 파일명 (예: 'problems_한화_스마일펫보험.json')
    """
    base_filename = os.path.splitext(os.path.basename(full_policy_text_file_path))[0]
    return f"{prefix}{base_filename}{extension}"

In [32]:
# 자동 파일명 생성
step2_results_file_name = generate_step2_result_filename(full_policy_text_file_path)

In [33]:
STEP2_RESULTS_SAVE_DIR = '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results' # 예시 경로

# 전체 저장 경로
step2_results_save_path = os.path.join(STEP2_RESULTS_SAVE_DIR, step2_results_file_name)


In [34]:
# --- Step 2 결과 저장 경로 지정 ---
# Google Drive 내에서 Step 2 결과를 저장할 폴더 경로
# 이 폴더가 없다면 자동으로 생성됩니다.
STEP2_RESULTS_SAVE_DIR = '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results' # 예시 경로
# 전체 저장 경로
step2_results_save_path = os.path.join(STEP2_RESULTS_SAVE_DIR, step2_results_file_name)
print(step2_results_save_path)


/content/drive/MyDrive/PROJECT/cn_api_project/step2_results/problems_삼성화재_반려묘보험_의기냥냥.json


In [35]:
# 저장할 데이터 확인 (all_problem_candidates_from_chunks 변수에 결과가 담겨 있는지)
# run_problem_identification_step 함수 실행 결과를 담았던 변수 이름을 확인하세요.
# 이전 코드에서 all_problem_candidates_from_chunks 변수에 결과가 담겼습니다.
data_to_save = all_problem_candidates_from_chunks if 'all_problem_candidates_from_chunks' in locals() and all_problem_candidates_from_chunks else []

print(data_to_save[:100])

[{'id': 1, '간단_표시용_요약': '고지의무 위반 시 불리한 해석', '문제점_요약': '고지의무 위반 시 보험사의 해지권 과다', '발생_가능_시나리오': '계약자가 경미한 병력을 고지하지 않은 것을 이유로 보험사에서 계약을 해지함', '근거_및_추론과정': "약관 제18조에서는 '고지 의무 위반의 효과'로 보험사의 해지권을 명시하고 있음. 금감원 분쟁사례집에서는 경미한 병력 미고지로 인해 보험사의 해지권 남용이 발생한 사례가 다수 존재함.", '관련_약관_원문_표현': '계약자 또는 피보험자가 고의 또는 중대한 과실로 제1항을 위반한 경우 회사는 계약을 해지할 수 있다.', '문제점_유형': ['고지 의무/통지 의무', '법령/판례와의 불일치'], '예상_소비자_영향': '계약 해지 및 보험금 미지급', '참고_자료_출처': ['금감원 분쟁사례집 2023 p.28', '보험업감독규정 제9조'], '근거_신뢰도': '신뢰'}, {'id': 2, '간단_표시용_요약': '모호한 표현으로 인한 소비자 혼란', '문제점_요약': '보험금 지급 제외 사유의 모호성', '발생_가능_시나리오': '소비자가 상해로 입원 후 보험금을 청구했더니 보험금 지급 제외 사유가 모호해 결과를 예측할 수 없었음', '근거_및_추론과정': "약관 제5조에서는 '보험금을 지급하지 않는 사유'에 관한 문구가 매우 포괄적이며 모호함. 예를 들어 '회사의 판단에 따라 보험금을 지급하지 않을 수 있다'는 표현이 있음. 이는 금감원 분쟁사례집 2022-4, p. 73의 사례에서도 문제가 되었음.", '관련_약관_원문_표현': '회사의 판단에 따라 보험금을 지급하지 않을 수 있다.', '문제점_유형': ['모호한 표현', '보상 제외 조건의 불명확성'], '예상_소비자_영향': '보험금 지급 지연 또는 미지급', '참고_자료_출처': ['금감원 분쟁사례집 2022-4, p. 73'], '근거_신뢰도': '주의'}, {'id': 3, '간단_표시용_요약': '자연재해로 인한 피해 보장 제외

In [36]:
print("\n--- Step 2 결과 (잠재적 문제점 후보 목록) Google Drive에 저장 시작 ---")

if data_to_save:
    try:
        # 저장 디렉토리 없으면 생성
        if not os.path.exists(STEP2_RESULTS_SAVE_DIR):
            os.makedirs(STEP2_RESULTS_SAVE_DIR, exist_ok=True)
            print(f"저장 디렉토리 생성: '{STEP2_RESULTS_SAVE_DIR}'")
        else:
            print(f"저장 디렉토리 확인: '{STEP2_RESULTS_SAVE_DIR}'")

        # JSON 파일 저장
        with open(step2_results_save_path, 'w', encoding='utf-8') as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=2)

        print(f"✅ Step 2 결과 저장 완료!")
        print(f"📄 저장 경로: {step2_results_save_path}")
        print(f"📌 저장된 문제점 후보 개수: {len(data_to_save)}")

    except Exception as e:
        print("❌ 저장 중 오류 발생!")
        print("오류 내용:", e)
        print("Google Drive 마운트 상태나 경로 권한을 확인하세요.")
else:
    print("⚠️ Step 2 분석 결과가 비어 있어 저장할 내용이 없습니다.")
    print("이전 분석 단계가 정상적으로 실행되었는지 확인하세요.")

print("--- Step 2 저장 코드 실행 완료 ---\n")


--- Step 2 결과 (잠재적 문제점 후보 목록) Google Drive에 저장 시작 ---
저장 디렉토리 확인: '/content/drive/MyDrive/PROJECT/cn_api_project/step2_results'
✅ Step 2 결과 저장 완료!
📄 저장 경로: /content/drive/MyDrive/PROJECT/cn_api_project/step2_results/problems_삼성화재_반려묘보험_의기냥냥.json
📌 저장된 문제점 후보 개수: 308
--- Step 2 저장 코드 실행 완료 ---

